# Upload Large Files v2

### Introduction

In this version we have an API endpoint which is integrated with a Lambda function that gets a pre-signed URL for an S3 bucket. The pre-signed URL is returned to the client who uploads a file (in this case an image) to the corresponding S3 bucket. To do this CORS is configured in the bucket so the client is allowed to upload the image.

After uploading the image, and EventNotification triggers a second Lambda function that is in charge of pre-processing the image and later store it in a different S3 bucket. The first S3 bucket contains "raw" data, while the second is for pre-processed images. In order for the pre-processing Lambda to `PUT` objects inside the bucket we need to configure:
1. The Lambda's layers: we must add the necessary Lambda layers required to perform our business logic.
2. The Lambda's role policy: we must add the necessary permissions so that the Lambda function can `PUT` objects inside the pre-processing bucket.
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "s3:PutObject",
                "s3:PutObjectAcl",
                "s3:GetObject",
                "s3:GetObjectAcl",
                "s3:DeleteObject"
            ],
            "Resource": [
                "arn:aws:s3:::my_bucket_name",
                "arn:aws:s3:::my_bucket_name/*"
            ]
        }
    ]
}
```
3. ~The Bucket's policy~: we must configure the bucket policy so that the Lambda can `PUT` objects inside it.
```
{
    "Version": "2012-10-17",
    "Id": "Policy1674139292765",
    "Statement": [
        {
            "Sid": "Stmt1674139290697",
            "Effect": "Allow",
            "Principal": {
                "AWS": "my_role_arn"
            },
            "Action": "s3:*",
            "Resource": "arn:aws:s3:::my_bucket_name"
        }
    ]
}
```
**Note:** this was not required, I deleted the policy and still worked. Must research when bucket policies are required.

### Add object metadata

We need to add object metadata when we request the pre-signed URL. We need this because downstream the object will be stored inside the client's designated folder in S3. But, without knowing who the user/client is, how can we store the object inside his folder? We need **user metadata**. The user metadata is passed using boto3 in the following way:
```
s3_client = boto3.client('s3')
response = s3_client.generate_presigned_url('put_object',
                                            Params={'Bucket': BUCKET_NAME,
                                                    'Key': OBJECT_NAME,
                                                    'ContentType': "application/jpeg",
                                                    'Metadata': {"id":"user-1"} # add user metadata
                                            },
                                            ExpiresIn=EXPIRATION)
```                                      

### Allow client to pass user metadata 

To allow the client to pass his metadata we need to change our HTTP method that invokes the `get_signed_url` function from a `GET` to a `POST` since `GET` methods are not able to pass a `body` with data.

### Documentation

- [Lambda policy for S3 video 1](https://www.youtube.com/watch?v=vXiZO1c5Sk0)
- [Lambda policy for S3 video 2](https://www.youtube.com/watch?v=MgQDeKwTnDQ)
- [Lambda layer repository](https://github.com/keithrozario/Klayers):
    - `numpy`: arn:aws:lambda:us-east-1:770693421928:layer:Klayers-p39-numpy:10
    - `pillow`: arn:aws:lambda:us-east-1:770693421928:layer:Klayers-p39-pillow:1
- [Add object metadata using pre-signed URLs](https://stackoverflow.com/questions/53649888/aws-s3-presigned-url-with-metadata)   
 